# Import the library

In [92]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Load the dataset

In [93]:

#import raw dataset
header_trans = ['marketplace','chain','contract','tx_hash','tx_index','block_number','order_hash','topic_hash','price','amount','token_id','token_contract','seller_address','buyer_address','created_at','updated_at','payment_token','eth_price','usd_price','time','gas_fee','activity','log_index','realized_pnl','unrealized_pnl','pnl_eth_price','pnl_amount','id']
transaction_data = pd.read_csv('transaction.csv', names=header_trans, header = None)

header_items = ['contract_address','token_id','property','value','chain','numeric_value','created_at','updated_at']
item_properties = pd.read_csv('item_properties.csv', names=header_items, header = None)

transaction_data.head()
item_properties.head()

/var/folders/gr/zfwfwf1d5m53t7q5y_455xsc0000gn/T/ipykernel_1606/453729303.py:3: DtypeWarning: Columns (17,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  transaction_data = pd.read_csv('transaction.csv', names=header_trans, header = None)


,contract_address,token_id,property,value,chain,numeric_value,created_at,updated_at
0,0x231d3559aa848bf10366fb9868590f01d34bf240,1,Background,Light Gray,ethereum,NaN,2023-05-31 03:43:22.445043,2023-05-31 03:43:22.445043
1,0x231d3559aa848bf10366fb9868590f01d34bf240,1,Clothes,Red/Camo Hoodie Vest,ethereum,NaN,2023-05-31 03:43:22.445043,2023-05-31 03:43:22.445043
2,0x231d3559aa848bf10366fb9868590f01d34bf240,1,Ear,Silver Cuffs,ethereum,NaN,2023-05-31 03:43:22.445043,2023-05-31 03:43:22.445043
3,0x231d3559aa848bf10366fb9868590f01d34bf240,1,Eye,Shaved Brow,ethereum,NaN,2023-05-31 03:43:22.445043,2023-05-31 03:43:22.445043
4,0x231d3559aa848bf10366fb9868590f01d34bf240,1,FaceAcc,White Bandaid,ethereum,NaN,2023-05-31 03:43:22.445043,2023-05-31 03:43:22.445043


In [94]:
#select appropriate columns for dataset
select_columns = ['token_contract','token_id','eth_price', 'time', 'gas_fee','tx_hash']
nft_contract = '0xed5af388653567af2f388e6224dc7c4b3241c544' #to test
dataset = transaction_data[select_columns]
dataset = dataset[dataset['token_contract'] == nft_contract]

dataset.info()


<class 'pandas.core.frame.DataFrame'>
Index: 51818 entries, 22 to 2831868
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   token_contract  51818 non-null  object 
 1   token_id        51818 non-null  object 
 2   eth_price       51778 non-null  object 
 3   time            51818 non-null  object 
 4   gas_fee         51818 non-null  float64
 5   tx_hash         51818 non-null  object 
dtypes: float64(1), object(5)
memory usage: 2.8+ MB


# Data cleaning and processing

## Change inappropriate datatype for this dataset

In [95]:
#change data type
dataset['eth_price'] = dataset['eth_price'].astype(float)/1e18
dataset['gas_fee'] = dataset['gas_fee']/1e18
dataset['time'] = pd.to_datetime(dataset['time'])
dataset['daySold'] = dataset['time'].dt.day
dataset['monthSold'] = dataset['time'].dt.month
dataset['yearSold'] = dataset['time'].dt.year
dataset['token_id'] = dataset['token_id'].astype(int)
#drop na
dataset.dropna(inplace=True)

dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51778 entries, 22 to 2831868
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   token_contract  51778 non-null  object        
 1   token_id        51778 non-null  int64         
 2   eth_price       51778 non-null  float64       
 3   time            51778 non-null  datetime64[ns]
 4   gas_fee         51778 non-null  float64       
 5   tx_hash         51778 non-null  object        
 6   daySold         51778 non-null  int32         
 7   monthSold       51778 non-null  int32         
 8   yearSold        51778 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(3), int64(1), object(2)
memory usage: 3.4+ MB


## Adjust item_properties data format to join with the main dataset

In [96]:
item_properties_converted = item_properties.pivot(index=['contract_address', 'token_id'], columns='property', values='value').reset_index()
data = pd.merge(item_properties_converted, dataset, how = "right",left_on=["contract_address", "token_id"], right_on=["token_contract","token_id"])
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51778 entries, 0 to 51777
Data columns (total 72 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   contract_address  51778 non-null  object        
 1   token_id          51778 non-null  int64         
 2   3D                0 non-null      object        
 3   ApeCoin Staked    0 non-null      object        
 4   Artist            0 non-null      object        
 5   Aura              0 non-null      object        
 6   Back              0 non-null      object        
 7   Background        51778 non-null  object        
 8   Bonus             0 non-null      object        
 9   Boost             0 non-null      object        
 10  Card              0 non-null      object        
 11  Class             0 non-null      object        
 12  Clothes           0 non-null      object        
 13  Clothing          51762 non-null  object        
 14  Collab            0 no

In [97]:
#drop columns having all null data
columns_with_all_null_data = data.columns[data.isna().all()]
data.drop(columns_with_all_null_data, axis=1, inplace=True)

data.head()

,contract_address,token_id,Background,Clothing,Ear,Eyes,Face,Hair,Headgear,Mouth,...,Special,Type,token_contract,eth_price,time,gas_fee,tx_hash,daySold,monthSold,yearSold
0,0xed5af388653567af2f388e6224dc7c4b3241c544,4220,Off White D,Dress Shirt,Spiked Ring,Careless,NaN,Indigo Long,NaN,Toothpick,...,NaN,Human,0xed5af388653567af2f388e6224dc7c4b3241c544,11.50,2022-08-02 04:09:41,0.001641,0xa19426dc29b2927a68efb4ee553cd2a254ac5fe6eebe...,2,8,2022
1,0xed5af388653567af2f388e6224dc7c4b3241c544,8680,Off White C,Frog Kigurumi,NaN,Daydreaming,NaN,Magenta Pigtails,NaN,Grass,...,NaN,Human,0xed5af388653567af2f388e6224dc7c4b3241c544,12.00,2022-08-02 03:03:10,0.001368,0x2a2fdada863be952c9320c137d983e1e1a9c9da6dc07...,2,8,2022
2,0xed5af388653567af2f388e6224dc7c4b3241c544,7944,Off White B,Black Qipao,NaN,Calm,NaN,Blue Bun,NaN,Smile,...,NaN,Human,0xed5af388653567af2f388e6224dc7c4b3241c544,14.61,2023-05-08 15:39:11,0.038899,0x9e073c83173f9d06912e4b6a3600fc0b9b6eb5274c17...,8,5,2023
3,0xed5af388653567af2f388e6224dc7c4b3241c544,2475,Off White C,Black T-Shirt,NaN,Meditating,Kabuki Facepaint,Long Dreadlocks,NaN,Not Bad,...,NaN,Human,0xed5af388653567af2f388e6224dc7c4b3241c544,14.48,2023-05-08 15:35:35,0.037571,0x7bea9c7fbb4c899f20f0c5ff033600b6a1ae14f54647...,8,5,2023
4,0xed5af388653567af2f388e6224dc7c4b3241c544,1445,Red,Rain Jacket,NaN,Relaxed,Bandaid,Black Hairband,Beret,Toothpick,...,NaN,Human,0xed5af388653567af2f388e6224dc7c4b3241c544,6.00,2022-08-14 11:39:28,0.001067,0x93c158ede33219ac9cce400e69833d78245374c71629...,14,8,2022


In [98]:
#replace null value with "Missing word"
data.fillna('NaN', inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51778 entries, 0 to 51777
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   contract_address  51778 non-null  object        
 1   token_id          51778 non-null  int64         
 2   Background        51778 non-null  object        
 3   Clothing          51778 non-null  object        
 4   Ear               51778 non-null  object        
 5   Eyes              51778 non-null  object        
 6   Face              51778 non-null  object        
 7   Hair              51778 non-null  object        
 8   Headgear          51778 non-null  object        
 9   Mouth             51778 non-null  object        
 10  Neck              51778 non-null  object        
 11  Offhand           51778 non-null  object        
 12  Special           51778 non-null  object        
 13  Type              51778 non-null  object        
 14  token_contract    5177

# Model building

In [99]:
X = data.drop(['eth_price', 'contract_address', 'token_id', 'tx_hash','time'], axis = 1)
y = data['eth_price']

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

## Categorical Variables Processing

In [107]:
# Select all objects columns
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)

Categorical variables:
['Background', 'Clothing', 'Ear', 'Eyes', 'Face', 'Hair', 'Headgear', 'Mouth', 'Neck', 'Offhand', 'Special', 'Type', 'token_contract']


In [108]:
# Apply one-hot encoder to each column with categorical data
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
col_X_train = pd.DataFrame(encoder.fit_transform(X_train[object_cols]))
col_X_valid = pd.DataFrame(encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
col_X_train.index = X_train.index
col_X_valid.index = X_valid.index


# # Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# # Add one-hot encoding data to numerical features

final_X_train = pd.concat([col_X_train, num_X_train], axis=1)
final_X_valid = pd.concat([col_X_valid, num_X_valid], axis=1)

# # Ensure all columns have string type
final_X_train.columns = final_X_train.columns.astype(str)
final_X_valid.columns = final_X_valid.columns.astype(str)


## Check dataset scoring

In [106]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder

my_pi

In [105]:
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(final_X_train, final_X_valid, y_train, y_valid))


MAE from Approach 3 (One-Hot Encoding):
1.079627167552003
